![alt text](data/cover.jpg "Title")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns; 
from scipy import stats
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
train = pd.read_csv("data/train_clean.csv", sep=',')
test = pd.read_csv("data/test_clean.csv", sep =',')

## Regression Logistique : Selection des Variables Explicatives

In [5]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2